<a href="https://colab.research.google.com/github/waigisteve/AccelerateDevGitHubCopilot/blob/main/Data_Analysis_Copilot_20251022_040708.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📊 Data Analysis Copilot

**Goal:** Develop an AI-powered agent that helps users analyze datasets and generate insights using natural language queries.

**Key Features:**
1. Convert natural language to SQL/Pandas queries (few-shot prompting)
2. Perform statistical analysis with Chain-of-Thought reasoning
3. Recommend data visualizations automatically
4. Detect anomalies and explain them
5. Suggest predictive modeling approaches

**Tech Stack:** Python, LangChain, Pandas, Plotly, Scikit-learn/Statsmodels, OpenAI/Gemini/Claude APIs

**Deployment:** Google Colab for interactive execution, public GitHub repository for collaboration

## Architecture & Workflow
User Input → AI Agent (GPT/Gemini/Claude) → Pandas/SQL → Analysis → Visualization → Predictive Modeling → Output

**Architectural Diagram:**
```
+-----------------+       +--------------------------+       +-------------------+
| User Natural    |  -->  | AI Agent (GPT/Gemini/Claude) | --> | Pandas/SQL Engine |
| Language Input  |       +--------------------------+       +-------------------+
+-----------------+                     |
                                         v
                            +-------------------------+
                            | Statistical Analysis &  |
                            | Anomaly Detection      |
                            +-------------------------+
                                         |
                                         v
                            +-------------------------+
                            | Visualization Module    |
                            +-------------------------+
                                         |
                                         v
                            +-------------------------+
                            | Predictive Modeling      |
                            +-------------------------+
                                         |
                                         v
                            +-------------------------+
                            | User Output/UI           |
                            +-------------------------+
```

In [3]:
# =========================================================
# 🧹 CLEAN INSTALL – Fix LangChain + OpenAI Compatibility
# =========================================================
# Works on: Python 3.12 (Colab) + OpenAI SDK v1.51+

!pip uninstall -y langchain langchain-core langchain-openai langchain-experimental langchain-community openai
!pip install --upgrade pip

# 🧩 Install compatible, flexible versions (verified 100% on Colab)
!pip install "langchain>=0.3.0,<0.4.0" "langchain-openai>=0.2.0,<0.4.0" "langchain-community>=0.3.0,<0.4.0" openai pandas

# ✅ Check what was actually installed
!pip show langchain langchain-core langchain-openai openai | grep Version


Found existing installation: langchain 0.3.27
Uninstalling langchain-0.3.27:
  Successfully uninstalled langchain-0.3.27
Found existing installation: langchain-core 0.3.79
Uninstalling langchain-core-0.3.79:
  Successfully uninstalled langchain-core-0.3.79
Found existing installation: openai 1.109.1
Uninstalling openai-1.109.1:
  Successfully uninstalled openai-1.109.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 29.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 17.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.8 MB/s  0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
   ━━━━━━━━━━━━━━━━━━━━━

In [4]:
!pip install -U langchain==0.3.27 langchain-core==0.3.79 langchain-openai==0.3.35 langchain-community==0.3.31 langchain-experimental==0.3.2 openai==2.6.1 tiktoken --quiet


In [5]:
# --- Step 2: Load OpenAI API Key from Colab Secrets ---
from google.colab import userdata
import os

# Load secret key
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

# Verify key is loaded
print("✅ API Key detected" if os.getenv("OPENAI_API_KEY") else "❌ API Key not found")


✅ API Key detected


In [6]:
# --- Imports ---
from langchain_openai import ChatOpenAI
from langchain_experimental.agents import create_pandas_dataframe_agent
import pandas as pd
import os

# --- Setup API Key (if not already set in environment) ---
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

# --- Sample DataFrame ---
df = pd.DataFrame({
    "Name": ["Stephen", "Mary", "John"],
    "Age": [34, 28, 45],
    "City": ["Nairobi", "Kisumu", "Mombasa"]
})

# --- Initialize LLM ---
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# --- Create the DataFrame Agent (allow dangerous code) ---
agent = create_pandas_dataframe_agent(
    llm,
    df,
    verbose=True,
    allow_dangerous_code=True   # 👈 This fixes the error
)

# --- Ask a question ---
response = agent.invoke("Who is the oldest person and what city are they from?")
print("\n💡 Response:", response)




> Entering new AgentExecutor chain...
Thought: To find the oldest person in the dataframe, I need to identify the row with the maximum age and then extract the corresponding name and city from that row.
Action: I will use the `idxmax()` function to find the index of the maximum age and then retrieve the name and city from that index.
Action Input: `oldest_index = df['Age'].idxmax(); oldest_person = df.loc[oldest_index, ['Name', 'City']]`I will use the `idxmax()` function to find the index of the maximum age and then retrieve the name and city from that index. is not a valid tool, try one of [python_repl_ast].Thought: I need to execute the command to find the index of the oldest person and then retrieve their name and city from the dataframe.
Action: I will execute the command to find the oldest person.
Action Input: `oldest_index = df['Age'].idxmax(); oldest_person = df.loc[oldest_index, ['Name', 'City']]`I will execute the command to find the oldest person. is not a valid tool, try 

In [9]:
# =========================================================
# 🤖 Smart Data Analysis Agent (Stage 2+)
# =========================================================
# Features:
# - Natural language → Pandas execution
# - Statistical analysis reasoning
# - Automatic Plotly visualizations
# - Works with your uploaded dataset (CSV, Excel, JSON)
# =========================================================

# --- Imports ---
from langchain_openai import ChatOpenAI
from langchain_experimental.tools.python.tool import PythonAstREPLTool
from langchain.agents import initialize_agent, AgentType
import pandas as pd
import plotly.express as px
from google.colab import files
import io

# --- Upload dataset ---
print("📁 Upload your dataset (CSV, Excel, or JSON):")
uploaded = files.upload()

# --- Auto-detect and load file ---
for filename, filecontent in uploaded.items():
    if filename.endswith(".csv"):
        df = pd.read_csv(io.BytesIO(filecontent))
    elif filename.endswith(".xlsx"):
        df = pd.read_excel(io.BytesIO(filecontent))
    elif filename.endswith(".json"):
        df = pd.read_json(io.BytesIO(filecontent))
    else:
        raise ValueError("Unsupported file format. Please upload CSV, Excel, or JSON.")

print(f"\n✅ Loaded file: {filename}")
print(f"📄 Shape: {df.shape}")
print("🔍 Preview:")
display(df.head())

# =========================================================
# 🧠 Create Smart Data Agent
# =========================================================
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Add Python execution tool for real data operations
python_tool = PythonAstREPLTool()

agent = initialize_agent(
    tools=[python_tool],
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
)

# Provide dataframe context
context = {"df": df}

# --- Quick Data Summary ---
print("\n📈 Quick Data Summary:")
print(df.describe(include='all').T)

# =========================================================
# 💬 Ask a Natural Language Question
# =========================================================
print("\n💬 Example questions you can ask:")
print(" - What are the top 5 categories by sales?")
print(" - Which country has the highest population?")
print(" - Show correlation between revenue and profit.")
print(" - List countries with names longer than 10 letters.")
print(" - Detect outliers in numeric columns.")
print(" - Suggest a chart for this dataset.")

query = input("\n🤖 Ask your question about the dataset: ")

# Build the agent prompt
prompt = f"""
You are analyzing a pandas dataframe called df.
Answer the question using Python and Pandas operations if needed.
Return the final answer or a concise explanation.
Question: {query}
"""

response = agent.invoke({"input": prompt, **context})

print("\n🧠 Response:", response["output"])

# =========================================================
# 🎨 Visualization Recommendation + Plotly
# =========================================================
def recommend_and_plot(df):
    print("\n🎨 Auto Visualization Suggestions:")
    numeric_cols = df.select_dtypes(include=['number']).columns
    if len(numeric_cols) >= 2:
        print(f"Plotting scatter: {numeric_cols[0]} vs {numeric_cols[1]}")
        fig = px.scatter(df, x=numeric_cols[0], y=numeric_cols[1], title="Scatter Plot")
        fig.show()
    elif len(numeric_cols) == 1:
        print(f"Plotting histogram for {numeric_cols[0]}")
        fig = px.histogram(df, x=numeric_cols[0], title="Distribution")
        fig.show()
    else:
        print("No numeric columns detected for visualization.")

recommend_and_plot(df)

print("\n✅ Stage 2+ Complete: You can now explore and query your dataset interactively!")


📁 Upload your dataset (CSV, Excel, or JSON):


Saving states_by_country.csv to states_by_country (2).csv

✅ Loaded file: states_by_country (2).csv
📄 Shape: (4989, 4)
🔍 Preview:


,State Code,State Name,Country Code,Country Name
0,BDS,Badakhshan,AF,Afghanistan
1,BDG,Badghis,AF,Afghanistan
2,BGL,Baghlan,AF,Afghanistan
3,BAL,Balkh,AF,Afghanistan
4,BAM,Bamyan,AF,Afghanistan


/tmp/ipython-input-2782586499.py:48: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(



📈 Quick Data Summary:
             count unique               top freq
State Code    4984   1677                 5   65
State Name    4989   4898  Western Province    5
Country Code  4975    200                GB  247
Country Name  4989    201    United Kingdom  247

💬 Example questions you can ask:
 - What are the top 5 categories by sales?
 - Which country has the highest population?
 - Show correlation between revenue and profit.
 - List countries with names longer than 10 letters.
 - Detect outliers in numeric columns.
 - Suggest a chart for this dataset.

🤖 Ask your question about the dataset: can you derive all countries that haveless that 10 counties and present them in descending order limit  20 ?


> Entering new AgentExecutor chain...
To derive all countries that have less than 10 counties from the dataframe `df`, I need to first check the structure of the dataframe to identify the relevant columns. Typically, I would look for a column that represents countries and another t

In [ ]:
# -------------------------------
# 0️⃣ Install required packages if needed
# -------------------------------
# !pip install pandas matplotlib seaborn scikit-learn langchain openai

# -------------------------------
# 1️⃣ Imports
# -------------------------------
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

from langchain.agents import Tool, initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI

# -------------------------------
# 2️⃣ Load Dataset
# -------------------------------
df = pd.read_csv("states_by_country.csv")
print("✅ Dataset loaded")
print(df.head())

# -------------------------------
# 3️⃣ Predictive Model Helper
# -------------------------------
def suggest_predictive_model():
    df_encoded = df.copy()
    for col in df_encoded.select_dtypes(include='object').columns:
        df_encoded[col] = LabelEncoder().fit_transform(df_encoded[col])
    numeric_cols = df_encoded.select_dtypes(include='number').columns
    if len(numeric_cols) >= 2:
        return "📈 Try Linear Regression with one numeric column as target."
    else:
        return "🤖 Mostly categorical; consider Decision Trees or Random Forest."

# -------------------------------
# 4️⃣ Python REPL Tool
# -------------------------------
def python_repl_tool(code: str):
    try:
        local_vars = {"df": df, "sns": sns, "plt": plt, "suggest_predictive_model": suggest_predictive_model}
        exec(code, globals(), local_vars)
        return str(list(local_vars.values())[-1])
    except Exception as e:
        return f"❌ Error: {e}"

tools = [
    Tool(
        name="python_repl_ast",
        func=python_repl_tool,
        description="Executes Python code on dataframe df. Use print() to see output."
    )
]

# -------------------------------
# 5️⃣ Initialize LLM
# -------------------------------
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# -------------------------------
# 6️⃣ Create Natural Language Agent
# -------------------------------
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True
)

# -------------------------------
# 7️⃣ Example Queries
# -------------------------------
example_queries = [
    "Show the first 5 countries starting with 'E'",
    "Show top 5 countries by number of states",
    "Suggest a predictive model for this dataset"
]

print("\n--- Running Example Queries ---\n")
for q in example_queries:
    print(f"\n📝 Query: {q}")
    output = agent.run(q)
    print(output)

# -------------------------------
# 8️⃣ Interactive Mode
# -------------------------------
print("\n--- Interactive Mode ---")
print("Type 'exit' to quit.")

while True:
    user_input = input("\nAsk a question: ")
    if user_input.lower() == "exit":
        break
    output = agent.run(user_input)
    print(output)


✅ Dataset loaded
  State Code  State Name Country Code Country Name
0        BDS  Badakhshan           AF  Afghanistan
1        BDG     Badghis           AF  Afghanistan
2        BGL     Baghlan           AF  Afghanistan
3        BAL       Balkh           AF  Afghanistan
4        BAM      Bamyan           AF  Afghanistan

--- Running Example Queries ---


📝 Query: Show the first 5 countries starting with 'E'


> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `countries = ['Egypt', 'Ecuador', 'El Salvador', 'Estonia', 'Eritrea', 'Ethiopia', 'Equatorial Guinea', 'Eswatini']
first_five_e_countries = [country for country in countries if country.startswith('E')][:5]
first_five_e_countries`


['Egypt', 'Ecuador', 'El Salvador', 'Estonia', 'Eritrea']The first 5 countries starting with 'E' are:

1. Egypt
2. Ecuador
3. El Salvador
4. Estonia
5. Eritrea

> Finished chain.
The first 5 countries starting with 'E' are:

1. Egypt
2. Ecuador
3. El Salvador
4. Estonia
5. Eritrea